In [7]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #

import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())

# Add the Scripts directory to Python path so we can import modules
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

In [8]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #
    
# Add functions here from Scripts folder
from download_ao_indices import ReturnAOFetch
from enso_indices import ReturnENSOFetch
from sea_ice_indices import CalculateIceIndex

# Add data here from Data folder
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.rtf")
ao_data_filepath = os.path.join(project_root, "Data", "monthly.ao.index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")
study_sites = ["Stebbins", "Hooper Bay"]

# Generate a list of years from 2000 to 2022 - select years
years_of_data = list(range(2000, 2023))

In [9]:
# ------------------------------------------------ #
#                  MAIN CODE                      #
# ------------------------------------------------ #

# Pull in ENSO data as dictionary of Datetime:Values on a MONTHLY scale from all years of data
# Format of this dictionary should go from March of year-1 to March of year (e.g. year 2000 is dictionary M1999-M2000) for each year
enso_indices = ReturnENSOFetch(years_of_data, enso_data_filepath)

print(enso_indices)

# Pull in AO data as dictionary of Datetime:Values on a MONTHLY scale from all years of data
# Format of this dictionary should go from March of year-1 to March of year (e.g. year 2000 is dictionary M1999-M2000) for each year
ao_indices = ReturnAOFetch(years_of_data, ao_data_filepath)

print(ao_indices)

# Find ice index, returned as a dictionary of Datetime:Index
ice_indices = CalculateIceIndex(years_of_data, ice_data_filepath, study_sites)

print(ice_indices)


TypeError: ReturnENSOFetch() takes 1 positional argument but 2 were given

In [10]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Placeholders)    #
# ------------------------------------------------ #

# Placeholder for correlation calculations (actual functions need to be defined)
# ao_correlation_values = CalculateAOCorrelation(ao_indices, ice_indices)
# enso_correlation_values = CalculateENSOCorrelation(enso_indices, ice_indices)

# Placeholder for charting (actual functions need to be defined)
# ChartENSOValues(enso_correlation_values)
# ChartAOValues(ao_correlation_values)
